In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import loadmat
import tensorflow as tf
import keras
import cv2

from keras import layers, regularizers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.optimizers import Adam

config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 4} )
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

Using TensorFlow backend.


## Load DVS data

In [153]:
# Load data
X = np.zeros((800, 180, 240, 2))
for i in range(800):
    frame_inc = cv2.imread("Data_DVS_superimpose/" + str(i) + "_inc.png", cv2.IMREAD_GRAYSCALE)
    frame_dec = cv2.imread("Data_DVS_superimpose/" + str(i) + "_dec.png", cv2.IMREAD_GRAYSCALE)
    X[i,:,:,0] = frame_inc / 255
    X[i,:,:,1] = frame_dec / 255

Y = np.zeros((800, 3))
yvals = pd.read_excel('Data_DVS_superimpose/data_trial_new_new.xlsx',sheet_name = 'Sheet1',nrows = 400)
y_orig = yvals.values[:,1].reshape(yvals.values[:,1].shape[0],1) # y.shape = (400,1)
y_aug = np.absolute(y_orig - 2)
y = np.concatenate((y_orig, y_aug), axis = 0)
for i in range(y.shape[0]):
    Y[i, int(y[i,0])] = 1
    
X_train = X[0:700, :]
Y_train = Y[0:700, :]
X_test = X[700: 750, :]
Y_test = Y[700: 750, :]
X_dev = X[750: 800, :]
Y_dev = Y[750: 800, :]

## Load pixel data

In [3]:
# RGB
# X = np.zeros((800, 180, 240, 3))
# Grayscale
X = np.zeros((800, 180, 240, 1))
for i in range(800):
    # RGB
#     img = image.load_img("Data_pixel/" + str(i) + ".jpg", target_size = (180, 240)) 
    # grayscale
    img = image.load_img("Data_pixel/" + str(i) + ".jpg", color_mode='grayscale', target_size = (180, 240)) # grayscale
    img_array = image.img_to_array(img)
    X[i] = img_array / 255


Y = np.zeros((800, 3))
yvals = pd.read_excel('Data_DVS_superimpose/data_trial_new_new.xlsx',sheet_name = 'Sheet1',nrows = 400)
y_orig = yvals.values[:,1].reshape(yvals.values[:,1].shape[0],1) # y.shape = (400,1)
y_aug = np.absolute(y_orig - 2)
y = np.concatenate((y_orig, y_aug), axis = 0)
for i in range(y.shape[0]):
    Y[i, int(y[i,0])] = 1

X_train = X[0:700, :]
Y_train = Y[0:700, :]
X_test = X[700: 750, :]
Y_test = Y[700: 750, :]
X_dev = X[750: 800, :]
Y_dev = Y[750: 800, :]

## Define model

In [4]:
def cnn_model_keras(input_shape, drop_prob, lambd):
    """
    Using keras to quickly prototype.
    """
    
    # Define the input placeholder as a tensor with shape input_shape
    X_input = Input(input_shape)
    
    # first layer: 2@180x240 -> 6@90x120
    X = Conv2D(6, (20,20), strides = (2,2), padding='same', kernel_regularizer=regularizers.l2(lambd), name = "conv1")(X_input)
    X = Activation("relu")(X)
    X = Dropout(drop_prob, noise_shape=None, seed=None)(X)
    
    # max pooling: 6@90x120 -> 6@45x60
    X = MaxPooling2D((2,2), strides = (2,2), padding='valid')(X)
    
    # second layer: 6@45x60 -> 12@23x30
    X = Conv2D(12, (5,5), strides = (2,2), padding='same', kernel_regularizer=regularizers.l2(lambd), name = "conv2")(X)
    X = Activation("relu")(X)
    X = Dropout(drop_prob, noise_shape=None, seed=None)(X)
    
    # third layer: 12@23x30 -> 18@12x15
    X = Conv2D(18, (5,5), strides = (2,2), padding='same', kernel_regularizer=regularizers.l2(lambd), name = "conv3")(X)
    X = Activation("relu")(X)
    X = Dropout(drop_prob, noise_shape=None, seed=None)(X)
    
    # fourth layer: 18@12x15 -> 24@6x8
    X = Conv2D(24, (5,5), strides = (5,5), padding='same', kernel_regularizer=regularizers.l2(lambd), name = "conv4")(X)
    X = Activation("relu")(X)
    X = Dropout(drop_prob, noise_shape=None, seed=None)(X)
    
    # flatten: 36@8x10 -> 1152
    X = Flatten()(X)
    
    # third layer: 1152 -> 64
    X = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(lambd), name='fc1')(X)
    
    # fourth layer: 64 -> 3
    X = Dense(3, activation='softmax', kernel_regularizer=regularizers.l2(lambd), name='fc2')(X)
    
    
    model = Model(inputs = X_input, outputs = X, name = "NN_model_2_keras")
    
    return model

## Train model

In [61]:
# create the model
cnn_model = cnn_model_keras(X_train[0].shape, drop_prob = 0.2, lambd = 0.025)

# compile the model
cnn_model.compile(optimizer = 'adam', loss = "categorical_crossentropy", metrics = ["accuracy"])

# train the model
cnn_model.fit(x = X_train, y = Y_train, epochs = 1, batch_size = X_train.shape[0])

Epoch 1/1
700/700 [==============================] - 1s 1ms/step - loss: 0.4787 - acc: 0.9443


## Evaluate model

In [62]:
preds = cnn_model.evaluate(x = X_dev, y = Y_dev)
print()
print ("Loss = " + str(preds[0]))
print ("Dev Accuracy = " + str(preds[1]))

preds = cnn_model.evaluate(x = X_test, y = Y_test)
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))
#cnn_model.summary()

50/50 [==============================] - 0s 522us/step

Loss = 0.7263352108001709
Dev Accuracy = 0.8600000023841858
50/50 [==============================] - 0s 502us/step

Loss = 0.8513526034355163
Test Accuracy = 0.8400000023841858


## Error analysis

In [36]:
errors = []
for i in range(50):
    pred = cnn_model.evaluate(x = X_test[i:i+1], y = Y_test[i:i+1])
    if(pred[1] < 1):
        y_pred = cnn_model.predict(X_test[i:i+1])
        max_prob = np.amax(y_pred)
        y_pred[y_pred == max_prob] = 1
        y_pred[y_pred < max_prob] = 0
        errors.append((i, y_pred))

for i in range(len(errors)):
    print("index: " + str(700 + errors[i][0]))
    print("predicted: " + str(errors[i][1]))
    print("correct: " + str(Y_test[errors[i][0]]))

1/1 [==============================] - 0s 4ms/step
index: 710
predicted: [[0. 1. 0.]]
correct: [1. 0. 0.]
index: 716
predicted: [[1. 0. 0.]]
correct: [0. 0. 1.]
index: 717
predicted: [[0. 1. 0.]]
correct: [0. 0. 1.]
index: 718
predicted: [[1. 0. 0.]]
correct: [0. 1. 0.]
index: 720
predicted: [[1. 0. 0.]]
correct: [0. 1. 0.]
index: 747
predicted: [[0. 1. 0.]]
correct: [1. 0. 0.]


## Save and Load Models

In [48]:
cnn_model.save('models_class/cnn_model_GPU_pixGray')

In [ ]:
cnn_model = load_model('models_class/cnn_model_GPU_pixRGB')

## Visualisation of Kernels
The method is to randomly initialise an input image for a kernel, and define a loss function to be the average value of output of the kernel, then train on the input image by gradient ascent to maximise the loss function.

In [ ]:
from keras import backend as K
from scipy.misc import imsave
# filter num:  6               6                  12              18              24
# layer name: 'activation_5', 'max_pooling2d_2', 'activation_6', 'activation_7', 'activation_8'

layer_dict = dict([(layer.name, layer) for layer in cnn_model.layers])

# choose layer and specific filter to visualise
layer_name = 'max_pooling2d_2'
filter_index = 2

#
input_img = cnn_model.inputs[0]

# create a loss to maximise the activation of a layer
layer_output = layer_dict[layer_name].output
loss = K.mean(layer_output[:, :, :, filter_index])

# compute the gradient of the input picture wrt this loss
grads = K.gradients(loss, input_img)[0]

# normalization trick: we normalize the gradient
grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)

# this function returns the loss and grads given the input picture
iterate = K.function([input_img], [loss, grads])

# initialise a random input image, and update pixel values by training
input_img_data = np.random.random((1, 100, 100, 2))

# run gradient ascent
for i in range(20):
    loss_value, grads_value = iterate([input_img_data])
    input_img_data += grads_value * 1
    
img = input_img_data[0,:,:,0]
imsave('layers/%s_filter_%d.png' % (layer_name, filter_index), img)